# Tutorial - build LinearSVC model with sklearn

This tutorial demonstrates how to use the Sci-kit Learn (sklearn) package to build linearSVC model, rank features, and use the model for prediction. We will be using the Kaggle sentiment data again.

Note that sklearn actually provides two SVM algorithms: SVC and LinearSVC. 

The SVC module allows for choosing nonlinear kernels, and it uses one-vs-one strategy for multi-class classification.

The LinearSVC module uses the linear kernel, and it uses one-vs-all strategy for multi-class classification, so linearSVC is generally faster than SVC. Since linear kernel works better for text classification in general, this tutorial demonstrates how to use LinearSVC for text classification.

# Step 1: Read in data

In [1]:
# this step is the same as the NB script

# read in the training data
# the data set includes four columns: PhraseId, SentenceId, Phrase, Sentiment
# In this data set a sentence is further split into phrases 
# in order to build a sentiment classification model
# that can not only predict sentiment of sentences but also shorter phrases

# A data example:
# PhraseId SentenceId Phrase Sentiment
# 1 1 A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .1

# the Phrase column includes the training examples
# the Sentiment column includes the training labels
# "0" for very negative
# "1" for negative
# "2" for neutral
# "3" for positive
# "4" for very positive


import pandas as p
import numpy as np
train = p.read_csv("/Users/shivangi/Downloads/deception_data_converted_final(1).tsv", delimiter='\t')
# y = train['sentiment'].values
# y1 = train['lie'].values
# X = train['review'].values

In [12]:
train.loc[train['sentiment'] == 'n', 'Sentiments'] = 0
train.loc[train['sentiment'] == 'p', 'Sentiments'] = 1

In [13]:
train.head()

,lie,sentiment,review,Sentiments
0,f,n,"'Mike\'s Pizza High Point, NY Service was very...",0.0
1,f,n,'i really like this buffet restaurant in Marsh...,0.0
2,f,n,"'After I went shopping with some of my friend,...",0.0
3,f,n,'Olive Oil Garden was very disappointing. I ex...,0.0
4,f,n,'The Seven Heaven restaurant was never known f...,0.0


In [14]:
y = train['Sentiments'].values
y1 = train['lie'].values
X = train['review'].values

# Step 2: Split train/test data for hold-out test

In [15]:
# this step is the same as the NB script

# check the sklearn documentation for train_test_split
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# "test_size" : float, int, None, optional
# If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
# If int, represents the absolute number of test samples. 
# If None, the value is set to the complement of the train size. 
# By default, the value is set to 0.25. The default will change in version 0.21. It will remain 0.25 only if train_size is unspecified, otherwise it will complement the specified train_size.    

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

(73,) (73,) (19,) (19,)
'I went to Applebees (regrettably) once and it was a train-wreck. The server was in a terrible mood, the beers arrived after the dinner was delivered, the appetizer was wrong, food was bad, the check was wrong, and there were barely any other people inside! I was pretty baffled at how so many things just went wrong in the whole process, the restaurant wasn\'t even busy. '
0.0
'After I went shopping with some of my friend, we went to DODO restaurant for dinner. I found worm in one of the dishes .'
0.0


Sample output from the code above:

(93636,) (93636,) (62424,) (62424,)
almost in a class with that of Wilde
3
escape movie
2

# Step 2.1 Data Checking

In [16]:
# this step is the same as the NB script

# Check how many training examples in each category
# this is important to see whether the data set is balanced or skewed

unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)))

[[ 0.  1.]
 [35. 38.]]


The sample output shows that the data set is skewed with 47718/93636=51% "neutral" examples. All other categories are smaller.

{0, 1, 2, 3, 4}
[[    0  4141]
 [    1 16449]
 [    2 47718]
 [    3 19859]
 [    4  5469]]

# Step 3: Vectorization

In [17]:
# this step is the same as the NB script

# sklearn contains two vectorizers

# CountVectorizer can give you Boolean or TF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# TfidfVectorizer can give you TF or TFIDF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

# Read the sklearn documentation to understand all vectorization options

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# several commonly used vectorizer setting

#  unigram boolean vectorizer, set minimum document frequency to 5
unigram_bool_vectorizer = CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english')

#  unigram term frequency vectorizer, set minimum document frequency to 5
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, stop_words='english')

#  unigram and bigram term frequency vectorizer, set minimum document frequency to 5
gram12_count_vectorizer = CountVectorizer(encoding='latin-1', ngram_range=(1,2), min_df=5, stop_words='english')

#  unigram tfidf vectorizer, set minimum document frequency to 5
unigram_tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=5, stop_words='english')


## Step 3.1: Vectorize the training data

In [18]:
# this step is the same as the NB script

# The vectorizer can do "fit" and "transform"
# fit is a process to collect unique tokens into the vocabulary
# transform is a process to convert each document to vector based on the vocabulary
# These two processes can be done together using fit_transform(), or used individually: fit() or transform()

# fit vocabulary in training documents and transform the training documents into vectors
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)

# check the content of a document vector
print(X_train_vec.shape)
print(X_train_vec[0].toarray())

# check the size of the constructed vocabulary
print(len(unigram_count_vectorizer.vocabulary_))

# print out the first 10 items in the vocabulary
print(list(unigram_count_vectorizer.vocabulary_.items())[:10])

# check word index in vocabulary
print(unigram_count_vectorizer.vocabulary_.get('imaginative'))

(73, 70)
[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 2 0]]
70
[('went', 68), ('terrible', 59), ('dinner', 16), ('food', 23), ('bad', 3), ('people', 45), ('just', 31), ('restaurant', 50), ('wasn', 67), ('friends', 26)]
None


Sample output:

(93636, 11967)
[[0 0 0 ..., 0 0 0]]
11967
[('imaginative', 5224), ('tom', 10809), ('smiling', 9708), ('easy', 3310), ('diversity', 3060), ('impossibly', 5279), ('buy', 1458), ('sentiments', 9305), ('households', 5095), ('deteriorates', 2843)]
5224

## Step 3.2: Vectorize the test data

In [19]:
# this step is the same as the NB script

# use the vocabulary constructed from the training data to vectorize the test data. 
# Therefore, use "transform" only, not "fit_transform", 
# otherwise "fit" would generate a new vocabulary from the test data

X_test_vec = unigram_count_vectorizer.transform(X_test)

# print out #examples and #features in the test set
print(X_test_vec.shape)

(19, 70)


Sample output:

(62424, 14324)

# Step 4: Train a LinearSVC classifier

In [20]:
# import the LinearSVC module
from sklearn.svm import LinearSVC

# initialize the LinearSVC model
svm_clf = LinearSVC(C=1)

# use the training data to train the model
svm_clf.fit(X_train_vec,y_train)

LinearSVC(C=1)

# Step 4.1 Interpret a trained LinearSVC model

In [21]:
## interpreting LinearSVC models
## http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

## LinearSVC uses a one-vs-all strategy to extend the binary SVM classifier to multi-class problems
## for the Kaggle sentiment classification problem, there are five categories 0,1,2,3,4 with 0 as very negative and 4 very positive
## LinearSVC builds five binary classifier, "very negative vs. others", "negative vs. others", "neutral vs. others", "positive vs. others", "very positive vs. others", 
## and then pick the most confident prediction as the final prediction.

## Linear SVC also ranks all features based on their contribution to distinguish the two concepts in each binary classifier
## For category "0" (very negative), get all features and their weights and sort them in increasing order
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_count_vectorizer.get_feature_names_out()))

## get the 10 features that are best indicators of very negative sentiment (they are at the bottom of the ranked list)
very_negative_10 = feature_ranks[-10:]
print("Very negative words")
for i in range(0, len(very_negative_10)):
    print(very_negative_10[i])
print()

## get 10 features that are least relevant to "very negative" sentiment (they are at the top of the ranked list)
not_very_negative_10 = feature_ranks[:10]
print("not very negative words")
for i in range(0, len(not_very_negative_10)):
    print(not_very_negative_10[i])
print()


Very negative words
(0.353880898499828, 'ask')
(0.3888561784431786, 'environment')
(0.45739450890051025, 'waiters')
(0.46621912582440517, 'great')
(0.5612568776517189, 'fresh')
(0.5899584909827561, 'delicious')
(0.6048687090652864, 'nice')
(0.6437211970258859, 'friendly')
(0.7862010356749219, 'need')
(0.9646190205193345, 'best')

not very negative words
(-0.7523672686874645, 'cold')
(-0.6548030711996905, 'dishes')
(-0.629116183817397, 'service')
(-0.5566384777295768, 'said')
(-0.533690262001584, 'went')
(-0.5216376439264832, 'meal')
(-0.5008968355572891, 'asked')
(-0.4879109847043264, 'ny')
(-0.46825282529132795, 'minutes')
(-0.4550554333175944, 'restaurant')



Sample output:

Very negative words
(1.5635285973159958, 'stunning')
(1.6005795078256047, 'astonish')
(1.610812924790558, 'refreshes')
(1.6148905161082145, 'flawless')
(1.6474647033330083, 'phenomenal')
(1.6506425169734038, 'masterful')
(1.67761558779458, 'masterfully')
(1.8781421016763864, 'glorious')
(1.9801881772566481, 'miraculous')
(2.0143251933052397, 'perfection')

not very negative words
(-2.3147454187985117, 'sacrifices')
(-1.8650987318574794, 'maintained')
(-1.8305667747223913, 'placed')
(-1.7974037295239631, 'argue')
(-1.6800998534753624, '19')
(-1.6684863939524339, 'homage')
(-1.6179084517399509, 'failure')
(-1.6088792786048403, 'breezy')
(-1.6059138072144292, 'bore')
(-1.5466693614369267, 'clone')

# Exercise A

In [22]:
# write code similar to the above sample code 
# and print out the 10 most indicative words for the "very positive" category and the "positive" category

# Your code starts here
feature_ranks = sorted(zip(svm_clf.coef_[1], unigram_count_vectorizer.get_feature_names_out()))

## get the 10 features that are best indicators of very negative sentiment (they are at the bottom of the ranked list)
very_positive_10 = feature_ranks[-10:]
print("Very positive words")
for i in range(0, len(very_positive_10)):
    print(very_positive_10[i])
print()

## get 10 features that are least relevant to "very negative" sentiment (they are at the top of the ranked list)
not_very_positive_10 = feature_ranks[:10]
print("not very positive words")
for i in range(0, len(not_very_positive_10)):
    print(not_very_positive_10[i])
print()
# Your code ends here

IndexError: index 1 is out of bounds for axis 0 with size 1

In [20]:
svm_clf.coef_.shape

(5, 11967)

In [16]:
X_train_vec

<93636x11967 sparse matrix of type '<class 'numpy.int64'>'
	with 334671 stored elements in Compressed Sparse Row format>

In [12]:
y_train

array([3, 2, 3, ..., 2, 3, 2])

Sample output

(1.5635285973159958, 'stunning')
(1.6005795078256047, 'astonish')
(1.610812924790558, 'refreshes')
(1.6148905161082145, 'flawless')
(1.6474647033330083, 'phenomenal')
(1.6506425169734038, 'masterful')
(1.67761558779458, 'masterfully')
(1.8781421016763864, 'glorious')
(1.9801881772566481, 'miraculous')
(2.0143251933052397, 'perfection')

# Step 5: Test the LinearSVC classifier

In [11]:
# test the classifier on the test data set, print accuracy score

svm_clf.score(X_test_vec,y_test)



0.6236864026656415

In [12]:
# print confusion matrix and classification report

from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2,3,4])
print(cm)
print()

from sklearn.metrics import classification_report
target_names = ['0','1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names))

[[  918  1221   697    82    13]
 [  701  4080  5504   514    25]
 [  195  2106 27081  2310   172]
 [   34   396  6048  5533  1057]
 [    3    51   590  1772  1321]]

              precision    recall  f1-score   support

           0       0.50      0.31      0.38      2931
           1       0.52      0.38      0.44     10824
           2       0.68      0.85      0.75     31864
           3       0.54      0.42      0.48     13068
           4       0.51      0.35      0.42      3737

    accuracy                           0.62     62424
   macro avg       0.55      0.46      0.49     62424
weighted avg       0.60      0.62      0.60     62424



# Step 5.1 Interpret the prediction result

In [15]:
## get the confidence scores for all test examples from each of the five binary classifiers
svm_confidence_scores = svm_clf.decision_function(X_test_vec)
## get the confidence score for the first test example
print(svm_confidence_scores[0])

## sample output: array([-1.05306321, -0.62746206,  0.31074854, -0.89709483, -1.08343089]
## because the confidence score is the highest for category 2, 
## the prediction should be 2. 

## Confirm by printing out the actual prediction
print(y_test[0])
print(X[0])

[-1.01718403 -0.50760026  0.2233101  -0.97514739 -1.24718853]
2
A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .


In [16]:
# output prediction probs
# https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html#sphx-glr-auto-examples-calibration-plot-calibration-curve-py

from sklearn.calibration import CalibratedClassifierCV
svm_calibrated = CalibratedClassifierCV(svm_clf) 
svm_calibrated.fit(X_train_vec, y_train)
y_test_proba = svm_calibrated.predict_proba(X_test_vec)
y_test_proba[0]

/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([0.03909121, 0.20406249, 0.60822947, 0.11914359, 0.02947324])

# Exercise B

In [17]:
# The above sample code examines LinearSVC's prediction confidence for the first example.
# Now you are going to print out another example with index=99 (the 100th example), 
# and examine the LinearSVC's prediction confidence and its final prediction
# does this prediction seem correct to you?

# Your code starts here

# Your code ends here

# Step 5.2 Error Analysis

In [18]:
# print out specific type of error for further analysis

# print out the very positive examples that are mistakenly predicted as negative
# according to the confusion matrix, there should be 53 such examples
# note if you use a different vectorizer option, your result might be different

err_cnt = 0
for i in range(0, len(y_test)):
    if(y_test[i]==4 and y_pred[i]==0):
        print(X_test[i])
        err_cnt = err_cnt+1
print("errors:", err_cnt)

10 minutes into the film you 'll be white-knuckled and unable to look away .
the film is never dull
greatly impressed by the skill of the actors involved in the enterprise
errors: 3


# Exercise C

In [20]:
# write code to print out 
# the errors that very negative examples were mistakenly predicted as very positive?
# and the errors that very positive examples were mistakenly predicted as very negative?

# Try find lingustic patterns for these two types of errors
# Based on the above error analysis, what suggestions would you give to improve the current model?

# Your code starts here

# Your code ends here

# Step 6. Cross Validation

In [21]:
# Cross validation
# for more details see https://scikit-learn.org/stable/modules/cross_validation.html
# evaluation metrics provided by sklearn - https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

# if you need to output only one metric, such as accuracy or f1_macro, use the "cross_val_score" function

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

svm_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('svm', LinearSVC(C=1))])
scores = cross_val_score(svm_clf_pipe, X, y, cv=3, scoring='f1_macro')

/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [22]:
# if you need to output multiple metrics, such as accuracy and f1_macro, use the "cross_validate" function
from sklearn.model_selection import cross_validate

scoring = ['accuracy', 'f1_macro', 'f1_micro', 'precision_macro', 'recall_macro']
svm_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('svm', LinearSVC(C=1))])
scores = cross_validate(svm_clf_pipe, X, y, cv=3, scoring=scoring, return_train_score=True)

/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [23]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_f1_micro',
 'test_precision_macro',
 'test_recall_macro',
 'train_accuracy',
 'train_f1_macro',
 'train_f1_micro',
 'train_precision_macro',
 'train_recall_macro']

In [24]:
# retrieve scores from a metric
def get_metric_scores (scores, metric, train_or_test, verbose=False):
    metric_name = train_or_test + '_' + metric
    print(metric_name) 

    metric_scores = scores[metric_name]
    if (verbose == True):
        print(metric_scores)
    avg = sum(metric_scores) / len(metric_scores)
    print('average')
    avg_formatted = "{:.3f}".format(avg)
    print(avg_formatted)

In [25]:
#retrieve test accuracy scores
get_metric_scores(scores, 'accuracy', 'test', verbose=True)

test_accuracy
[0.57600923 0.56401384 0.56680123]
average
0.569


In [26]:
#retrieve training accuracy scores
get_metric_scores(scores, 'accuracy', 'train')

train_accuracy
average
0.738


In [27]:
# compare performance with different choice of C values

svm_clf_pipe2 = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('svm', LinearSVC(C=0.5))])
scores2 = cross_validate(svm_clf_pipe2, X, y, cv=3, scoring=scoring, return_train_score=True)

svm_clf_pipe3 = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('svm', LinearSVC(C=2))])
scores3 = cross_validate(svm_clf_pipe3, X, y, cv=3, scoring=scoring, return_train_score=True)

/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/byu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: Convergenc

In [28]:
# compare the effect of different C values
# C=1
print('C=1\n')
get_metric_scores(scores, 'accuracy', 'train')
get_metric_scores(scores, 'accuracy', 'test')

# C=0.5
print('\nC=0.5\n')
get_metric_scores(scores2, 'accuracy', 'train')
get_metric_scores(scores2, 'accuracy', 'test')

# C=2
print('\nC=2\n')
get_metric_scores(scores3, 'accuracy', 'train')
get_metric_scores(scores3, 'accuracy', 'test')



C=1

train_accuracy
average
0.738
test_accuracy
average
0.569

C=0.5

train_accuracy
average
0.732
test_accuracy
average
0.574

C=2

train_accuracy
average
0.741
test_accuracy
average
0.565


# Exercise D

In [29]:
# retrieve the f1_macro scores and compare the effect of C on f1_macro 

# Your code starts here

# Your code ends here